In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
csv_file_path = 'HDBPropertyInformation.csv' 
df = pd.read_csv(csv_file_path)
df.head()

In [ ]:
def get_postal_code(street_name, block):
    # Construct the search query for the OneMap API
    query = f"{block} {street_name}"
    print(query)
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={query}&returnGeom=Y&getAddrDetails=Y&pageNum=1"

    # Make the API request
    response = requests.get(url)
    data = response.json()

    # Extract postal code from the response
    if data["found"] > 0:
        print(data["results"][0]["POSTAL"])
        return data["results"][0]["POSTAL"]
    else:
        return None


In [ ]:
# Iterate through rows and get postal codes
postal_codes = []
for index, row in df.iterrows():
    street_name = row['street']
    block = row['blk_no']
    
    try:
        postal_code = get_postal_code(street_name, block)
    except Exception as e:
        print(f"Error processing address ({street_name}, {block}): {e}")
        postal_code = None  # or handle the error in a different way if needed

    postal_codes.append(postal_code)

In [ ]:
# Add the postal codes to the DataFrame
df['PostalCode'] = postal_codes

In [ ]:
# Save the updated DataFrame to a new CSV file
output_csv_path = 'output_file.csv'  # Replace with the desired output file path
df.to_csv(output_csv_path, index=False)
print("Postal codes added and saved to:", output_csv_path)